In [1]:
print("loading")
from IPython.display import display,HTML,clear_output
import IPython
from ipywidgets import widgets, Output
from ipywidgets.widgets import Dropdown,BoundedIntText,BoundedFloatText,Button,IntSlider,FloatSlider,HBox,VBox,Box,GridBox,Layout,Label,Tab,DatePicker,SelectionSlider
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fsolve
from time import sleep
import numpy as np
import traitlets as tl
from numpy import log,sqrt,exp,inf,nan,pi
import collections
import datetime
import grapher as gr
import pricer
import utils
import copy
import query
import time
base = 253

c = gr.theme()

loading


In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))
widgets.Tab().add_class('data_input')
widgets.HTML(c.data_style)

HTML(value='\n        <style>.p-TabBar-tab \n                { color:rgb(255,255,255) !important; background-c…

In [3]:
def clear_data():
    drd_strike.options = [0]
    drd_expiry.options = [0]
    
    fig_lhs.data = []
    fig_rhs.data = []
    fig_pnl.data = []
    fig_trm.data = []
    
    tbl_totals.data[0].cells.font.color=c.white 
    tbl_totals.data[0].header.values=["","","","OpenExp:",0,"",0,0,0,0,0,0,0,"",0,0]
    tbl_totals.data[0].cells.values=["","","","PnL:"    ,0,"",0,0,0,0,0,0,0,"",0,0]
    tbl_trades.data[0].cells.values=None
    tbl_params.data[0].cells.values=[["Open","Close"],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
    
def fun_load(change): 
    global cs
    
    clear_data()
    symbol = drd_symbol.value
    qdate = drd_date.value.strftime('%d/%m/%Y')
    qtime = drd_time.value[:5]
    
    ld = gr.LoadData(symbol,qdate,qtime)
    tl.directional_link((gr.prg_load, "val"), (prb_loading, "value"))
    
    cs = ld.start(sld_loc.value,sld_scale.value)
    pop_data()
    
    tl.directional_link((cs.prg_pay, "val"), (prb_payoff, "value"))
    tl.directional_link((cs.prg_trm, "val"), (prb_term, "value"))
    
    gr.prg_load.val = 0.
    
#-----------------------------------------------------------------------------------------------------------------------------------------------------

def fun_expiry(change):
    if drd_right.value=="Spot": drd_right.value="Call"
    i = drd_expiry.value
    
    drd_strike.options = cs.get_raw(i,"STR",0)
    
    sld_TNR.value = cs.fs["TNR"][i]
    sld_SPT.value = cs.fs["SPT"][i]
    sld_ATM.value = cs.fs["ATM"][i]
    sld_SKW.value = cs.fs["SKW"][i]
    sld_KRT.value = cs.fs["KRT"][i]
    sld_DIV.value = cs.fs["DIV"][i]
    sld_BOR.value = cs.fs["BOR"][i]
    sld_FRT.value = cs.fs["FRT"][i]
    sld_XH0.value = cs.fs[f"{drd_right.value[0]}H0"][i]
    sld_XH1.value = cs.fs[f"{drd_right.value[0]}H1"][i]
    sld_XKH.value = cs.fs[f"{drd_right.value[0]}KH"][i]
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    
    data[1][0] = cs.get_raw(i,"TNR")[0]
    data[1][1] = cs.get_raw(i,"TNR")[0]-sld_TNR.value
    data[2][0] = cs.get_raw(i,"SPT")[0]
    data[2][1] = cs.get_raw(i,"SPT")[0] * (1 + sld_SPT.value)
    data[3][0] = cs.get_raw(i,"ATM")[0]
    data[3][1] = cs.get_raw(i,"ATM")[0] * (1 + sld_ATM.value)
    data[4][0] = cs.get_raw(i,"SKW")[0]
    data[4][1] = cs.get_raw(i,"SKW")[0] * (1 + sld_SKW.value)
    data[5][0] = cs.get_raw(i,"KRT")[0]
    data[5][1] = cs.get_raw(i,"KRT")[0] * (1 + sld_KRT.value)
    data[6][0] = cs.get_raw(i,"DIV")[0]
    data[6][1] = cs.get_raw(i,"DIV")[0] * (1 + sld_DIV.value)
    data[7][0] = cs.get_raw(i,"BOR")[0]
    data[7][1] = cs.get_raw(i,"BOR")[0] * (1 + sld_BOR.value)
    data[8][0] = cs.get_raw(i,"FRT")[0]
    data[8][1] = cs.get_raw(i,"FRT")[0] * (1 + sld_FRT.value)
    data[9][0] = cs.get_raw(i,f"{drd_right.value[0]}H0")[0]
    data[9][1] = cs.get_raw(i,f"{drd_right.value[0]}H0")[0] * (1 + sld_XH0.value)
    data[10][0] = cs.get_raw(i,f"{drd_right.value[0]}H1")[0]
    data[10][1] = cs.get_raw(i,f"{drd_right.value[0]}H1")[0] * (1 + sld_XH1.value)
    data[11][0] = cs.get_raw(i,f"{drd_right.value[0]}KH")[0]
    data[11][1] = cs.get_raw(i,f"{drd_right.value[0]}KH")[0] * (1 + sld_XKH.value)
    data[12][0] = cs.get_raw(i,"YLD")[0]
    data[12][1] = cs.get_raw(i,"YLD")[0]
    data[13][0] = cs.get_raw(i,"CDD")[0]
    data[13][1] = cs.get_raw(i,"CDD")[0]-sld_TNR.value
    data[14][0] = cs.get_raw(i,"PHI")[0]
    data[14][1] = cs.get_raw(i,"PHI")[0]
    data[15][0] = cs.get_raw(i,"RHO")[0]
    data[15][1] = cs.get_raw(i,"RHO")[0]
    tbl_params.data[0].cells.values = data
    
def fun_TNR(change):    
    i = drd_expiry.value
    cs.fs["TNR"]=sld_TNR.value
    cs.fs["CDD"]=sld_TNR.value
    data=[[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[1][0]=cs.get_raw(i,"TNR")[0]
    data[1][1]=cs.get_raw(i,"TNR")[0]-sld_TNR.value
    data[13][0]=cs.get_raw(i,"CDD")[0]
    data[13][1]=cs.get_raw(i,"CDD")[0]-sld_TNR.value
    if cs.get_raw(i,"CDD")[0]-sld_TNR.value < 0: 
        data[6][1]=0
        data[12][1]=cs.get_raw(i,"BOR")[0]*(1 + sld_BOR.value)
    else:
        data[6][1]=cs.get_raw(i,"DIV")[0]*(1 + sld_DIV.value)
        data[12][1]=cs.get_raw(i,"YLD")[0]
    tbl_params.data[0].cells.values=data

def fun_SPT(change):
    fun_XKH(0)
    i = drd_expiry.value
    cs.fs["SPT"] = sld_SPT.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[2][0] = cs.get_raw(0,"SPT")[0]
    data[2][1] = cs.get_raw(0,"SPT")[0] * (1 + sld_SPT.value)    
    tbl_params.data[0].cells.values=data

def fun_ATM(change):
    i = drd_expiry.value
    cs.fs["ATM"][drd_expiry.value] = sld_ATM.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[3][0] = cs.get_raw(i,"ATM")[0]
    data[3][1] = cs.get_raw(i,"ATM")[0] * (1 + sld_ATM.value)
    TNR = data[1][1]/base
    ATM = data[3][1]
    SKW = data[4][1]
    KRT = data[5][1]
    theta,phi,rho = pricer.SSVI().jw2raw(TNR,ATM,SKW,KRT)
    data[14][1] = phi
    data[15][1] = rho
    tbl_params.data[0].cells.values=data
    
def fun_SKW(change):
    i = drd_expiry.value
    cs.fs["SKW"][drd_expiry.value] = sld_SKW.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[4][0] = cs.get_raw(i,"SKW")[0]
    data[4][1] = cs.get_raw(i,"SKW")[0] * (1 + sld_SKW.value)
    TNR = data[1][1]/base
    ATM = data[3][1]
    SKW = data[4][1]
    KRT = data[5][1]
    theta,phi,rho = pricer.SSVI().jw2raw(TNR,ATM,SKW,KRT)
    data[14][1] = phi
    data[15][1] = rho
    tbl_params.data[0].cells.values=data
    
def fun_KRT(change):
    i = drd_expiry.value
    cs.fs["KRT"][drd_expiry.value] = sld_KRT.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[5][0] = cs.get_raw(i,"KRT")[0]
    data[5][1] = cs.get_raw(i,"KRT")[0] * (1+sld_KRT.value)
    TNR = data[1][1]/base
    ATM = data[3][1]
    SKW = data[4][1]
    KRT = data[5][1]
    theta,phi,rho = pricer.SSVI().jw2raw(TNR,ATM,SKW,KRT)
    data[14][1] = phi
    data[15][1] = rho
    tbl_params.data[0].cells.values=data
    
def fun_DIV(change):
    i = drd_expiry.value
    cs.fs["DIV"][drd_expiry.value] = sld_DIV.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[6][0] = cs.get_raw(i,"DIV")[0]
    data[6][1] = cs.get_raw(i,"DIV")[0] * (1 + sld_DIV.value)
    tbl_params.data[0].cells.values=data
    
def fun_BOR(change):
    i = drd_expiry.value
    cs.fs["BOR"][drd_expiry.value] = sld_BOR.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[7][0] = cs.get_raw(i,"BOR")[0]
    data[7][1] = cs.get_raw(i,"BOR")[0] * (1 + sld_BOR.value)
    tbl_params.data[0].cells.values=data
    
def fun_FRT(change):
    i = drd_expiry.value
    cs.fs["FRT"][drd_expiry.value] = sld_FRT.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[8][0] = cs.get_raw(i,"FRT")[0]
    data[8][1] = cs.get_raw(i,"FRT")[0] * (1 + sld_FRT.value)
    tbl_params.data[0].cells.values=data
    
def fun_XH0(change):
    i = drd_expiry.value
    R = drd_right.value[0]
    cs.fs[f"CH0"][drd_expiry.value] = sld_XH0.value
    cs.fs[f"PH0"][drd_expiry.value] = sld_XH0.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[9][0] = cs.get_raw(i,f"{R}H0")[0]
    data[9][1] = cs.get_raw(i,f"{R}H0")[0] * (1 + sld_XH0.value)
    tbl_params.data[0].cells.values=data
    
def fun_XH1(change):
    i = drd_expiry.value
    R = drd_right.value[0]
    cs.fs["CH1"][drd_expiry.value] = sld_XH1.value
    cs.fs["PH1"][drd_expiry.value] = sld_XH1.value
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[10][0] = cs.get_raw(i,f"{R}H1")[0]
    data[10][1] = cs.get_raw(i,f"{R}H1")[0] * (1 + sld_XH1.value)
    tbl_params.data[0].cells.values=data
    
def fun_XKH(change):
    i = drd_expiry.value
    R = drd_right.value[0]
    cs.fs["CKH"][drd_expiry.value] = (sld_XKH.value + sld_SPT.value)
    cs.fs["PKH"][drd_expiry.value] = (sld_XKH.value + sld_SPT.value)
    data = [[x for x in lst] for lst in tbl_params.data[0].cells.values]
    data[11][0] = cs.get_raw(i,f"{R}KH")[0]
    data[11][1] = cs.get_raw(i,f"{R}KH")[0] * (1 + (sld_XKH.value + sld_SPT.value))
    tbl_params.data[0].cells.values=data
    
def fun_right(change):
    if drd_right.value == "Call" or drd_right.value == "Put":
        fun_XH0(0)
        fun_XH1(0)
        fun_XKH(0)
        fun_fig_rhs({"new":drd_fig_rhs_yaxis.value})
        fun_fig_lhs({"new":drd_fig_lhs_yaxis.value})

def fun_clear(change):
    cs.ts = cs.pack_trd()
    tbl_totals.data[0].cells.font.color=c.white 
    tbl_totals.data[0].header.values=["","","","OpenExp:",0,"",0,0,0,0,0,0,0,"",0,0]
    tbl_totals.data[0].cells.values=["","","","PnL:"    ,0,"",0,0,0,0,0,0,0,"",0,0]
    tbl_trades.data[0].cells.values=None
    for x in range(len(fig_lhs.data)):
        idx = np.where(np.array(fig_lhs.data[x].marker.size)==7)[0]
        if len(idx)==0: continue
        fdt_s=fig_lhs.data[x]
        clr=[fdt_s.line.color]*len(fdt_s.marker.color)
        siz=[4]*len(fdt_s.marker.color)
        fdt_s.marker.color=clr
        fdt_s.marker.size=siz

def fun_reset(change):
    fun_clear(0)
    sld_TNR.value = 0
    sld_SPT.value = 0
    sld_ATM.value = 0
    sld_SKW.value = 0
    sld_KRT.value = 0
    sld_DIV.value = 0
    sld_BOR.value = 0
    sld_FRT.value = 0
    sld_XH0.value = 0
    sld_XH1.value = 0
    sld_XKH.value = 0
    cs.clear_factor()
    cs.pop(1)
    fun_expiry(0)
    fun_fig_rhs({"new":drd_fig_rhs_yaxis.value})
    fun_fig_lhs({"new":drd_fig_lhs_yaxis.value})

def fig_axis_update(fig,xcol,ycol,xaxis_name=None,yaxis_name=None,raw=True,yraw=None,only_open=False,payoff=None):
    if xaxis_name is not None: fig.update_layout(xaxis_title=xaxis_name) 
    if yaxis_name is not None: fig.update_layout(yaxis_title=yaxis_name)
    if raw == True:
        if payoff is not None:
            with fig_lhs.batch_update():
                idx = cs.groups+cs.groups+cs.groups
                ps = cs.rs[[xcol,payoff]][:,0]
                ps = np.sort(ps,order=[xcol])
                
                fig_lhs.data[idx].x = ps[xcol]
                fig_lhs.data[idx].y = ps[payoff]
        else:
            with fig_lhs.batch_update():
                idx = cs.groups+cs.groups+cs.groups
                fig_lhs.data[idx].visible = False
                
        for i in range(cs.groups):
            with fig.batch_update():
                fig.data[i].x = cs.get_raw(i,xcol,1)
                fig.data[i].y = cs.get_raw(i,ycol,1)
                if only_open == False:
                    if fig_lhs.data[i].visible == False: fig_lhs.data[i].visible = True
                    if fig_lhs.data[i+cs.groups+cs.groups].visible == False: fig_lhs.data[i+cs.groups+cs.groups].visible = True

                    fig.data[i+cs.groups].x = cs.get_raw(i,xcol,0)
                    fig.data[i+cs.groups].y = cs.get_raw(i,ycol,0)
                    fig.data[i+cs.groups+cs.groups].x = cs.get_raw(i,xcol,0)
                    fig.data[i+cs.groups+cs.groups].y = cs.get_raw(i,yraw,0)
                else:
                    fig_lhs.data[i].visible = False
                    fig_lhs.data[i+cs.groups+cs.groups].visible = False
                    
                    fig.data[i+cs.groups].x = cs.get_raw(i,xcol,0)
                    fig.data[i+cs.groups].y = cs.get_raw(i,ycol,0)
                    
    else:
        for i in range(cs.groups):
            with fig.batch_update():
                fig.data[i].x = cs.get_smt(i,xcol,1)
                fig.data[i].y = cs.get_smt(i,ycol,1)
                fig.data[i+cs.groups].x = cs.get_smt(i,xcol,0)
                fig.data[i+cs.groups].y = cs.get_smt(i,ycol,0)
                fig.data[i+cs.groups+cs.groups].x = cs.get_smt(i,xcol,1,True)
                fig.data[i+cs.groups+cs.groups].y = cs.get_smt(i,ycol,1,True)       
                fig.data[i+cs.groups+cs.groups+cs.groups].x = cs.get_smt(i,xcol,0,True)
                fig.data[i+cs.groups+cs.groups+cs.groups].y = cs.get_smt(i,ycol,0,True)  
                
                
def fun_fig_lhs(change):
    selection = change["new"]
    if selection not in drd_fig_lhs_yaxis.options: return
    if selection != "Prices": tgl_fig_lhs_payoff.value = True
    if drd_right.value == "Spot": drd_right.value="Call"
    R = drd_right.value[0]
    xaxis = {"Strike":"STR","Moneyness":"MON","Delta":f"{R}DL","LogStrike":"LST"}
    xslct = drd_fig_lhs_xaxis.value
    xhead = xaxis[xslct]
    fig = fig_lhs
    if selection=="ImpVol": fig_axis_update(fig,xhead,"VOL",xslct,"ImpVol",raw=True,yraw="RVOL")  
    elif selection=="Prices": fig_axis_update(fig,xhead,f"{R}PX",xslct,f"{drd_right.value}Px",raw=True,yraw=f"R{R}MD",payoff=f"{R}PY")           
    elif selection=="Spreads": fig_axis_update(fig,xhead,f"{R}HS",xslct,f"{drd_right.value}Spread",raw=True,yraw=f"R{R}HS")
    elif selection=="ExercisePremium": fig_axis_update(fig,xhead,f"{R}EP",xslct,f"{drd_right.value}ExPremium",raw=True,yraw=f"R{R}EP")   
    elif selection=="PxErrors": fig_axis_update(fig,xhead,f"{R}ER",xslct,f"{drd_right.value}PxErrors",raw=True,only_open=True)
    elif selection=="VolErrors": fig_axis_update(fig,xhead,"VER",xslct,"VolErrors",raw=True,only_open=True)
        
def fun_fig_rhs(change):
    selection = change["new"]
    if selection not in drd_fig_rhs_yaxis.options: return
    if drd_right.value == "Spot": drd_right.value="Call"
    R = drd_right.value[0]
    xaxis = {"Strike":"STR","Moneyness":"MON","Delta":f"{R}DL","LogStrike":"LST"}
    xslct = drd_fig_rhs_xaxis.value
    xhead = xaxis[xslct]
    fig = fig_rhs
    if selection=="TotImpVar": fig_axis_update(fig,xhead,"VAR",xslct,"TotImpVar",raw=False)    
    elif selection=="Density": fig_axis_update(fig,xhead,"RND",xslct,"Density",raw=False) 
    elif selection=="ImpVol": fig_axis_update(fig,xhead,"VOL",xslct,"ImpVol",raw=False)    
    elif selection=="Delta": fig_axis_update(fig,xhead,f"{R}DL",xslct,f"{drd_right.value}Delta",raw=False)       
    elif selection=="Gamma": fig_axis_update(fig,xhead,"GAM",xslct,"Gamma",raw=False)    
    elif selection=="Vega": fig_axis_update(fig,xhead,"VEG",xslct,"Vega",raw=False)    
    elif selection=="Theta": fig_axis_update(fig,xhead,f"{R}TH",xslct,f"{drd_right.value}Theta",raw=False)    
    elif selection=="Rho": fig_axis_update(fig,xhead,f"{R}RH",xslct,f"{drd_right.value}Rho",raw=False)    
    elif selection=="Speed": fig_axis_update(fig,xhead,"SPD",xslct,"Speed",raw=False)    
    elif selection=="Vomma": fig_axis_update(fig,xhead,"VOM",xslct,"Vomma",raw=False)    
    elif selection=="Vanna": fig_axis_update(fig,xhead,"VAN",xslct,"Vanna",raw=False)    
    elif selection=="Charm": fig_axis_update(fig,xhead,f"{R}CH",xslct,f"{drd_right.value}Charm",raw=False)    
        
def fun_apply(change):
    cs.pop(1)
    fun_fig_rhs({"new":drd_fig_rhs_yaxis.value})
    fun_fig_lhs({"new":drd_fig_lhs_yaxis.value})
    fun_expiry(0)
    pop_trades()
    
def highlight_points(points):
    fig_data = fig_lhs.data[points.trace_index]
    c = np.array(fig_data.marker.color)
    s = np.array(fig_data.marker.size)
    c[points.point_inds],s[points.point_inds]="red",7
    with fig_lhs.batch_update(): 
        fig_data.marker.color=c
        fig_data.marker.size=s

def fun_select(trace,points,selector):
    if len(points.xs)==0:
        return
    else:
        if drd_right.value != "Spot": highlight_points(points)
        rights = {"Call":1,"Put":0,"Spot":-1}
        if drd_side.value == "Lift":
                side = 1 if str(points.trace_name[-3:]) == "opn" else -1
        elif drd_side.value == "Hit":
                side = -1 if str(points.trace_name[-3:]) == "opn" else 1
        LEG = 0 if str(points.trace_name[-3:]) == "opn" else 1
        IDX = int(points.trace_name[0]) if drd_right.value != "Spot" else -1
        STR = points.xs[0] if drd_right.value != "Spot" else 0
        RGT = rights[drd_right.value]
        SIZ = sld_size.value * 100 * side if drd_right.value != "Spot" else sld_size.value * side
        cs.append_trd(RGT,LEG,IDX,STR,SIZ)
        pop_trades()
    
def highlight_points(points):
    fig_data = fig_lhs.data[points.trace_index]
    c = np.array(fig_data.marker.color)
    s = np.array(fig_data.marker.size)
    c[points.point_inds],s[points.point_inds]="red",7
    with fig_lhs.batch_update(): 
        fig_data.marker.color=c
        fig_data.marker.size=s
        
def fun_raw(change):
    selection = change["new"]
    if selection == True:
        with fig_lhs.batch_update():
            for i in range(cs.groups):
                idx = i+cs.groups+cs.groups
                fig_lhs.data[idx].visible = False
    elif selection == False:
        with fig_lhs.batch_update():
            for i in range(cs.groups): 
                idx = i+cs.groups+cs.groups
                fig_lhs.data[idx].visible = True if fig_lhs.data[i].visible==True else 'legendonly'
                
def fun_atmf(change):
    selection = change["new"]
    if selection == True:
        with fig_rhs.batch_update():
            for i in range(cs.groups): 
                idx1 = i+cs.groups+cs.groups
                idx2 = i+cs.groups+cs.groups+cs.groups
                fig_rhs.data[idx1].visible = False
                fig_rhs.data[idx2].visible = False
    elif selection == False:
        with fig_rhs.batch_update():
            for i in range(cs.groups): 
                idx1 = i+cs.groups+cs.groups
                idx2 = i+cs.groups+cs.groups+cs.groups
                fig_rhs.data[idx1].visible = True
                fig_rhs.data[idx2].visible = True

def fun_payoff(change):
    tgl_payoff_hs.value = False
    tgl_payoff_eep.value = False
    cs.build_pay()
    STR = (np.array(cs.ts["STR"][np.where(cs.ts["LEG"]==1)[0]])/cs.get_raw(0,"SPT",0)[0])-1
    with fig_pnl.batch_update():
        fig_pnl.data[0].x = cs.ps["PCTM"]
        fig_pnl.data[0].y = cs.ps["LQPX"] + cs.ps["SPTR"] + cs.ps["LQHS"] + cs.ps["LQEP"]
        cs.prg_pay.val += 0.05
        fig_pnl.data[1].x = cs.ps["PCTM"]
        fig_pnl.data[1].y = cs.ps["EXPX"] + cs.ps["SPTR"] + cs.ps["EXHS"] + cs.ps["EXEP"]
        cs.prg_pay.val += 0.05
        fig_pnl.data[2].x = STR
        fig_pnl.data[2].y = [0]*len(STR)
    sleep(0.3)
    cs.prg_pay.val = 0.
    
def fun_payoff_legs(change):
    selection = change["new"]
    if selection == True:
        with fig_pnl.batch_update():
            fig_pnl.data[2].visible = False
    elif selection == False:
        with fig_pnl.batch_update():
            fig_pnl.data[2].visible = True
            
def fun_payoff_hs(change):
    selection = change["new"]
    if selection == True:
        with fig_pnl.batch_update():
            fig_pnl.data[0].y = fig_pnl.data[0].y - cs.ps["LQHS"]
            fig_pnl.data[1].y = fig_pnl.data[1].y - cs.ps["EXHS"]          
    elif selection == False:
        with fig_pnl.batch_update():
            fig_pnl.data[0].y = fig_pnl.data[0].y + cs.ps["LQHS"]
            fig_pnl.data[1].y = fig_pnl.data[1].y + cs.ps["EXHS"]

def fun_payoff_eep(change):
    selection = change["new"]
    if selection == True:
        with fig_pnl.batch_update():
            fig_pnl.data[0].y = fig_pnl.data[0].y - cs.ps["LQEP"]
            fig_pnl.data[1].y = fig_pnl.data[1].y - cs.ps["EXEP"]          
    elif selection == False:
        with fig_pnl.batch_update():
            fig_pnl.data[0].y = fig_pnl.data[0].y + cs.ps["LQEP"]
            fig_pnl.data[1].y = fig_pnl.data[1].y + cs.ps["EXEP"]

def fun_term_hs(change):
    selection = change["new"]
    if selection == True:
        with fig_trm.batch_update():
            fig_trm.data[0].y = fig_trm.data[0].y - cs.ns["LQHS"]
            fig_trm.data[1].y = fig_trm.data[1].y - cs.ns["EXHS"]          
    elif selection == False:
        with fig_trm.batch_update():
            fig_trm.data[0].y = fig_trm.data[0].y + cs.ns["LQHS"]
            fig_trm.data[1].y = fig_trm.data[1].y + cs.ns["EXHS"]

def fun_term_eep(change):
    selection = change["new"]
    if selection == True:
        with fig_trm.batch_update():
            fig_trm.data[0].y = fig_trm.data[0].y - cs.ns["LQEP"]
            fig_trm.data[1].y = fig_trm.data[1].y - cs.ns["EXEP"]          
    elif selection == False:
        with fig_trm.batch_update():
            fig_trm.data[0].y = fig_trm.data[0].y + cs.ns["LQEP"]
            fig_trm.data[1].y = fig_trm.data[1].y + cs.ns["EXEP"]
            
def fun_term(change):
    tgl_term_hs.value = False
    tgl_term_eep.value = False
    jump = True if tgl_term_exjump.value == False else False
    cs.build_trm(jump)
    with fig_trm.batch_update():
        fig_trm.data[0].x = cs.ns["DPAS"]
        fig_trm.data[0].y = cs.ns["LQPX"] + cs.ns["SPTR"] + cs.ns["LQHS"] + cs.ns["LQEP"]
        cs.prg_trm.val += 0.05
        fig_trm.data[1].x = cs.ns["DPAS"]
        fig_trm.data[1].y = cs.ns["EXPX"] + cs.ns["SPTR"] + cs.ns["EXHS"] + cs.ns["EXEP"]
        cs.prg_trm.val += 0.05
    sleep(0.3)
    cs.prg_trm.val = 0.
            
def highlight_points(points,via_graph=True):
    if via_graph == True:
        fig_data = fig_lhs.data[points.trace_index]
        c = np.array(fig_data.marker.color)
        s = np.array(fig_data.marker.size)
        c[points.point_inds],s[points.point_inds]="red",7
    else:
        fig_data = fig_lhs.data[points["trace_idx"]]
        c = np.array(fig_data.marker.color)
        s = np.array(fig_data.marker.size)
        c[points["point_inds"]],s[points["point_inds"]]="red",7        
    with fig_lhs.batch_update(): 
        fig_data.marker.color=c
        fig_data.marker.size=s
        
def fun_trade(change):
    rights = {"Call":1,"Put":0,"Spot":-1}
    points = dict()
    if drd_right.value != "Spot":
        STR = drd_strike.value
        points["trace_idx"] = drd_expiry.value + cs.groups if drd_leg.value=="Open" else drd_expiry.value
        fig_data = fig_lhs.data[points["trace_idx"]]
        points["point_inds"] = np.where(fig_data.x==STR)
        highlight_points(points,via_graph=False)
    else:
        STR = 0
    side = 1 if drd_side.value == "Lift" else -1
    IDX = drd_expiry.value if drd_right.value != "Spot" else -1    
    LEG = 0 if drd_leg.value == "Open" else 1
    RGT = rights[drd_right.value]
    SIZ = sld_size.value * 100 * side if drd_right.value != "Spot" else sld_size.value * side
    cs.append_trd(RGT,LEG,IDX,STR,SIZ)
    pop_trades()    
        
def pop_trades():
    st = cs.build_trd()
    trd = cs.unpack_trd(st)
    tbl_trades.data[0].cells.values = trd
    trd[0] = ["rgb(255,127,80)" if x=="Close" else "rgb(0,255,255)" for x in trd[0]]
    trd[1:] = [c.white]*len(trd[1:])
    tbl_trades.data[0].cells.font.color = trd
    if len(st["LEG"][np.where(st["LEG"]==0)[0]]) == len(st["LEG"][np.where(st["LEG"]==1)[0]]):
        exp_row,pnl_row,pnl_clr = cs.totals(st)
        tbl_totals.data[0].cells.values = pnl_row
        tbl_totals.data[0].cells.font.color = pnl_clr
        tbl_totals.data[0].header.values = exp_row
            
def fun_fig_lhs_xaxis(change):
    selection = change["new"]
    if selection not in drd_fig_lhs_xaxis.options: 
        return
    else:
        fun_fig_lhs({"new":drd_fig_lhs_yaxis.value})
    
def fun_fig_rhs_xaxis(change):
    selection = change["new"]
    if selection not in drd_fig_rhs_xaxis.options: 
        return
    else:
        fun_fig_rhs({"new":drd_fig_rhs_yaxis.value})

def fun_liq(change):
    cs.liquidate_trd()
    pop_trades()

def fun_date(change):
    if type(change["new"]) is not datetime.date: return
    
def pop_data():
    fun_expiry(0)
    drd_expiry.options = list(range(cs.groups))
    drd_strike.options = cs.get_raw(0,"STR")
    tgl_fig_lhs_payoff.value = True
    
    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)
        gr.add_trace(fig_lhs,cs.get_raw(i,"STR"),cs.get_raw(i,"VOL",1),f"{i}:cls",i,"markers+lines",f"rgba(255,127,80,{opacity})") #Close

    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)    
        gr.add_trace(fig_lhs,cs.get_raw(i,"STR"),cs.get_raw(i,"VOL",0),f"{i}:opn",i,"markers+lines",f"rgba(0,255,255,{opacity})") #Open

    gr.prg_load.val += 0.1
    
    for i in fig_lhs.data: 
        i.on_click(fun_select)
        i.on_selection(fun_select)

    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)    
        gr.add_trace(fig_lhs,cs.get_raw(i,"STR"),cs.get_raw(i,"RVOL",0),f"{i}:raw",i,"markers",f"rgba(255,255,255,{opacity})") #Raw
    
    gr.add_trace(fig_lhs,[0,0],[0,0],"payoff",i,"lines","rgba(255,255,255,0.9)",legend=False,visible=False) #Payoff
        
    gr.prg_load.val += 0.1
    
    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)
        gr.add_trace(fig_rhs,cs.get_smt(i,"LST"),cs.get_smt(i,"RND"),f"{i}:cls","cls","lines",f"rgba(255,127,80,{opacity})") #Close

    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)
        gr.add_trace(fig_rhs,cs.get_smt(i,"LST"),cs.get_smt(i,"RND"),f"{i}:opn","opn","lines",f"rgba(0,255,255,{opacity})") #Open

    gr.prg_load.val += 0.1
    
    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)
        gr.add_trace(fig_rhs,cs.get_smt(i,"LST",1,True),cs.get_smt(i,"RND",1,True),f"{i}:ATMF cls","cls","markers",f"rgba(255,127,80,{opacity})",legend=False,marker_size=4) #ATMF Close

    for i in range(cs.groups): 
        opacity = round(1/(i+1),1)
        gr.add_trace(fig_rhs,cs.get_smt(i,"LST",0,True),cs.get_smt(i,"RND",0,True),f"{i}:ATMF opn","opn","markers",f"rgba(0,255,255,{opacity})",legend=False,marker_size=4) #ATMF Open
     
    gr.prg_load.val += 0.1
    
    gr.add_trace(fig_pnl,[-0.25,0.25],[0,0],"liqval",None,"lines","indianred")   #Total return from liquidation
    gr.add_trace(fig_pnl,[-0.25,0.25],[0,0],"payoff",None,"lines","gray")  #Spread cost
    gr.add_trace(fig_pnl,[0]*10,[0]*10,"contract",None,"markers","rgb(255,0,0)",legend=False,marker_size=6) 
    
    gr.prg_load.val += 0.1
    
    gr.add_trace(fig_trm,[0,max(cs.days_pass)],[0,0],"liqval",None,"lines","indianred")   #Total return from liquidation
    gr.add_trace(fig_trm,[0,max(cs.days_pass)],[0,0],"payoff",None,"lines","gray")  #Spread cost
    gr.add_trace(fig_trm,[cs.get_raw(0,"CDD",0)[0]+1],[0],f"ex-date","divs","markers","rgb(255,127,0)",legend=False,marker_size=6)
    for i in range(cs.groups): 
        gr.add_trace(fig_trm,[cs.get_raw(i,"TNR",0)[0]],[0],f"expiry {i}","expiry","markers","rgb(255,0,0)",legend=False,marker_size=6)
    
    drd_fig_lhs_yaxis.value = "ImpVol"
    drd_fig_rhs_yaxis.value = "Density"
    drd_fig_lhs_xaxis.value = "Strike"
    drd_fig_rhs_xaxis.value = "LogStrike"
    
    gr.prg_load.val += 0.1

In [4]:
main_width=1280
graph_height=370

fig_lhs = gr.create_figure("Strike","ImpVol","  Expiry:Leg",round(main_width/2)-6,graph_height,l=20,r=20,b=50,t=10)
fig_rhs = gr.create_figure("LogStrike","Density","  Expiry:Leg",round(main_width/2)-6,graph_height,l=20,r=20,b=50,t=10)
fig_pnl = gr.create_figure("dSpot","PnL"," ",round(main_width/2)-6,graph_height,l=20,r=20,b=50,t=10,yzeroline=True)
fig_trm = gr.create_figure("dTenor","PnL",None,round(main_width/2)-6,graph_height,l=20,r=20,b=50,t=10,yzeroline=True)
#--------------------------------------------------------------------------------------------------------------------------------------
tbl_params = gr.create_grid(
                            ["Leg","Tenor","SpotPx","AtmVol","AtmSkew","AtmKurt","CashDiv","ImpBor","FndRate","SprdMin","SprdSlope","SprdStrike","DivYld","CumDivDays","Convexity","SkewTerm"],
                            [["Open","Close"],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]],None,
                            [None,".0f",",.3f",",.3f",",.3f",",.3f",",.3f",",.3f",",.3f",",.3f",",.3f",",.3f",",.3f",".0f",",.3f",",.3f"],
                            main_width,98,30,30,l=2,r=20,b=3,t=3,
                            cell_font_colour=[["rgb(0,255,255)","rgb(255,127,80)"]]+[c.white]*15
                           )
tbl_trades = gr.create_grid(
                            ["Leg","Tenor","Side","Right","Size","Strike","MidPx","Delta","Gamma","Vega","Theta","Volga","Vanna","ImpVol","Spread/2","ExPremium"],[],None,
                            [None,None,None,None,",.0f",".2f",",.2f",".2f",".2f",".2f",".2f",".2f",".2f",".2f",".2f",".3f"],main_width,260,30,32,l=2,r=20,b=3,t=3,
                           )
tbl_totals = gr.create_grid(
                            ["","","","OpenExp:",0,"",0,0,0,0,0,0,0,"",0,0],
                            ["","","","PnL:"    ,0,"",0,0,0,0,0,0,0,"",0,0],
                            [None,None,None,None,".0f",None,",.2f",".2f",".2f",".2f",".2f",".2f",".2f",None,".2f",".2f"],
                            [None,None,None,None,".0f",None,",.2f",".2f",".2f",".2f",".2f",".2f",".2f",None,".2f",".2f"],
                            main_width,70,30,32,l=2,r=20,b=3,t=3,header_fill_colour=c.gray,header_font_colour=c.white
                           )
#--------------------------------------------------------------------------------------------------------------------------------------
sld_TNR = gr.factor_slider(0,90,1)
sld_SPT = gr.factor_slider(-1,1,0.01)
sld_ATM = gr.factor_slider(-1,1,0.01)
sld_SKW = gr.factor_slider(-10,10,0.1)
sld_KRT = gr.factor_slider(-1,2,0.1)
sld_DIV = gr.factor_slider(-1,1,0.01)
sld_BOR = gr.factor_slider(-1,10,0.1)
sld_FRT = gr.factor_slider(-1,10,0.1)
sld_XH0 = gr.factor_slider(-1,1,0.1)
sld_XH1 = gr.factor_slider(-1,1,0.1)
sld_XKH = gr.factor_slider(-1,1,0.1)
sld_size = gr.factor_slider(1,10000,1)

sld_loc = gr.factor_slider(-1,1,0.01)
sld_scale = gr.factor_slider(0,1,0.1)

btn_load = gr.add_button("Load")
btn_payoff = gr.add_button("RunPayoff")
btn_apply = gr.add_button("Apply") 
btn_reset = gr.add_button("Reset")
btn_clear = gr.add_button("Clear")
btn_term = gr.add_button("RunPayoff")
btn_liq = gr.add_button("Liquidate")
btn_trade = gr.add_button("Trade")

drd_date = DatePicker(disabled=False,layout=Layout(width='auto'),value=datetime.date(2020,9,25))
drd_expiry = gr.add_dropdown([0],0)
drd_side = gr.add_dropdown(["Lift","Hit"],"Lift")
drd_right = gr.add_dropdown(["Call","Put","Spot"],"Call")
drd_quotes = gr.add_dropdown(["Historical","Snapshot"],"Historical")
drd_symbol = gr.add_dropdown(gr.symbol_universe(),"JPM")
drd_time = gr.add_dropdown(gr.trading_times(drd_symbol.value),"20:40 UTC")
drd_leg = gr.add_dropdown(["Open","Close"],"Open")
drd_fig_lhs_yaxis = gr.add_dropdown(["ImpVol","Prices","Spreads","ExercisePremium","PxErrors","VolErrors"],"ImpVol")
drd_fig_rhs_yaxis = gr.add_dropdown(["Density","TotImpVar","ImpVol","Delta","Gamma","Vega","Theta","Rho","Speed","Vomma","Vanna","Charm"],"Density")
drd_fig_lhs_xaxis = gr.add_dropdown(["Strike","Moneyness","Delta","LogStrike"],"Strike")
drd_fig_rhs_xaxis = gr.add_dropdown(["Strike","Moneyness","Delta","LogStrike"],"LogStrike")
drd_strike = gr.add_dropdown([None],None)

tgl_fig_lhs_payoff = gr.add_toggle("Payoff")
tgl_payoff_eep = gr.add_toggle("ExPremium")
tgl_payoff_hs = gr.add_toggle("Spread")
tgl_payoff_legs = gr.add_toggle("Legs")
tgl_term_eep = gr.add_toggle("ExPremium")
tgl_term_hs = gr.add_toggle("Spread")
tgl_term_exjump = gr.add_toggle("ExDateJump")
tgl_raw = gr.add_toggle("Raw")
tgl_atm = gr.add_toggle("ATMF")

prb_term = gr.add_progress_bar()
prb_payoff = gr.add_progress_bar()
prb_loading = gr.add_progress_bar()

XX = widgets.Label(value="")
#--------------------------------------------------------------------------------------------------------------------------------------
sld_TNR.observe(fun_TNR)
sld_SPT.observe(fun_SPT)
sld_ATM.observe(fun_ATM)
sld_SKW.observe(fun_SKW)
sld_KRT.observe(fun_KRT)
sld_DIV.observe(fun_DIV)
sld_BOR.observe(fun_BOR)
sld_FRT.observe(fun_FRT)
sld_XH0.observe(fun_XH0)
sld_XH1.observe(fun_XH1)
sld_XKH.observe(fun_XKH)

drd_fig_lhs_yaxis.observe(fun_fig_lhs)
drd_fig_rhs_yaxis.observe(fun_fig_rhs)
drd_fig_lhs_xaxis.observe(fun_fig_lhs_xaxis)
drd_fig_rhs_xaxis.observe(fun_fig_rhs_xaxis)
drd_right.observe(fun_right)
drd_expiry.observe(fun_expiry)
drd_date.observe(fun_date)

btn_apply.on_click(fun_apply)
btn_reset.on_click(fun_reset)
btn_clear.on_click(fun_clear)
btn_load.on_click(fun_load)
btn_payoff.on_click(fun_payoff)
btn_liq.on_click(fun_liq)
btn_term.on_click(fun_term)
btn_trade.on_click(fun_trade)

def fun_fig_lhs_payoff(change):
    if drd_fig_lhs_yaxis.value != "Prices": return
    selection = change["new"]
    if selection == True:
        with fig_lhs.batch_update():
            idx = cs.groups+cs.groups+cs.groups
            fig_lhs.data[idx].visible = False
    elif selection == False:
        with fig_lhs.batch_update():
            idx = cs.groups+cs.groups+cs.groups
            fig_lhs.data[idx].visible = True


tgl_fig_lhs_payoff.observe(fun_fig_lhs_payoff)
tgl_atm.observe(fun_atmf)
tgl_raw.observe(fun_raw)
tgl_payoff_hs.observe(fun_payoff_hs)
tgl_payoff_eep.observe(fun_payoff_eep)
tgl_payoff_legs.observe(fun_payoff_legs)
tgl_term_hs.observe(fun_term_hs)
tgl_term_eep.observe(fun_term_eep)

#--------------------------------------------------------------------------------------------------------------------------------------
cols,rmargin = 16,.015
txt_len = ((1 - rmargin)/cols)*100
txt_tail = f"{txt_len}% "*12+f"{txt_len*2}% "*2
txt_main = f"{txt_len}% "*cols
txt_side = f"{((1 - (rmargin+(txt_len/100)))/10)*100}% "*5
txt_figs = txt_side + f"{txt_len}%" + txt_side

panel_1_children = [btn_apply,sld_TNR,sld_SPT,sld_ATM,sld_SKW,sld_KRT,sld_DIV,sld_BOR,sld_FRT,sld_XH0,sld_XH1,sld_XKH,drd_quotes,prb_loading]
panel_1 = GridBox(children=panel_1_children,layout=Layout(width='auto',grid_template_rows='auto',grid_template_columns=txt_tail))

panel_2_children = [drd_leg,drd_expiry,drd_side,drd_right,sld_size,drd_strike,btn_trade,btn_liq,btn_clear,btn_reset,btn_load,drd_symbol,drd_date,drd_time]
panel_2 = GridBox(children=panel_2_children,layout=Layout(width='auto',grid_template_rows='auto',grid_template_columns=txt_tail))


panel_3_children = [drd_fig_lhs_yaxis,drd_fig_lhs_xaxis,tgl_raw,tgl_fig_lhs_payoff,XX,XX,drd_fig_rhs_yaxis,drd_fig_rhs_xaxis,tgl_atm,sld_loc,sld_scale]
panel_3 = GridBox(children=panel_3_children,layout=Layout(width='auto',grid_template_rows='auto',grid_template_columns=txt_figs))

panel_4_children = [btn_payoff,tgl_payoff_eep,tgl_payoff_hs,tgl_payoff_legs,prb_payoff,XX,btn_term,tgl_term_eep,tgl_term_hs,tgl_term_exjump,prb_term]
panel_4 = GridBox(children=panel_4_children,layout=Layout(width='auto',grid_template_rows='auto',grid_template_columns=txt_figs))

graphs_upper=Box([fig_lhs,fig_rhs],layout=Layout(display='flex',flex_flow='row',align_items='flex-start',width='auto'))
graphs_lower=Box([fig_pnl,fig_trm],layout=Layout(display='flex',flex_flow='row',align_items='flex-start',width='auto'))
main = Box([tbl_params,panel_1,panel_2,tbl_trades,tbl_totals,panel_3,graphs_upper,panel_4,graphs_lower],
    layout=Layout(display='flex',flex_flow='column',align_items='stretch',width=f'{main_width+2}px'))

children = [main,main]
tab = widgets.Tab(children=children,layout=widgets.Layout(width=f'{main_width+20}px',margin='0 0 0 0'))
tab.set_title(0, "Main")
tab.set_title(1, "About")



tab

    'data': [{'cells': {'align': [left, left, left, left, left, lef…

    Tenor  Strike  LogStrike  Moneyness    RawVol    SmtVol  CallBid  CallAsk  \
0      16    60.0  -0.431873       0.64       NaN  0.677144    32.75    33.90   
1      16    65.0  -0.351830       0.70       NaN  0.606672    27.75    28.85   
2      16    70.0  -0.277722       0.75       NaN  0.541425    22.85    23.70   
3      16    75.0  -0.208729       0.80  0.478729  0.480682    17.80    18.90   
4      16    80.0  -0.144191       0.86       NaN  0.423860    13.00    13.70   
5      16    85.0  -0.083566       0.91  0.374791  0.370484     8.40     8.95   
6      16    90.0  -0.026408       0.96  0.348831  0.347901     4.65     4.85   
7      16    93.0   0.006382       1.00  0.342543  0.341890     2.97     3.10   
8      16    95.0   0.027659       1.02  0.340482  0.340515     2.12     2.22   
9      16    95.5   0.032909       1.02  0.340345  0.340470     1.94     2.04   
10     16    97.5   0.053635       1.04  0.340999  0.341391     1.34     1.43   
11     16    99.5   0.073940

KeyError: 0